In [1]:
import openrouteservice as ors
import folium
import folium.plugins as plugins
import random

client = ors.Client(key='5b3ce3597851110001cf6248facc383c642b420fa9818331b313169e')

In [2]:
import pandas as pd
# Read Data
nodes = pd.read_csv("SolutionData\\Nodes.csv")
truckRoute = pd.read_csv("SolutionData\\TruckRoute.csv")
droneRoute = pd.read_csv("SolutionData\\DroneRoute.csv")
nodes

,Index,X,Y,Type
0,0,28.797346,41.057196,TRUCK
1,1,28.844143,41.028388,UAVELIGIBLE
2,2,29.063855,40.968601,UAVELIGIBLE
3,3,29.102735,41.027243,UAVELIGIBLE
4,4,29.278667,40.968546,UAVELIGIBLE
5,5,28.683448,41.346898,UAVELIGIBLE
6,6,29.170114,40.895355,UAVELIGIBLE
7,7,29.088953,41.020695,UAVELIGIBLE
8,8,28.701336,41.000418,UAVELIGIBLE
9,9,28.705158,41.000202,TRUCK


In [4]:
m = folium.Map(location=[40.80533085, 29.442315], tiles='cartodbpositron', zoom_start=13)

# Add Nodes
droneEligableChance = 0.7
for index, row in nodes.iterrows():
    color = None
    if(row["Type"] == "UAVELIGIBLE"):
        color = "orange"
    else:
        color = "cyan"
    if(index == 0 or index == nodes.shape[0]-1):
        icon = plugins.BeautifyIcon(icon = "truck", iconShape="marker", backgroundColor="white")
    else:
        icon = plugins.BeautifyIcon(icon = "", iconShape="marker", number = row["Index"], backgroundColor=color)
    folium.Marker([row["Y"], row["X"]], icon=icon, popup= row["Index"],).add_to(m)
m

In [5]:
truckRoute

,Index,X,Y,CompletionTime
0,0,28.797346,41.057196,0.00
1,1,28.844143,41.028388,705.08
2,9,28.705158,41.000202,1786.77
3,8,28.701336,41.000418,1861.29
4,2,29.063855,40.968601,4288.43
5,6,29.170114,40.895355,5106.40
6,4,29.278667,40.968546,6389.04
7,7,29.088953,41.020695,7511.15
8,3,29.102735,41.027243,7773.29
9,5,28.683448,41.346898,11045.74


In [6]:
# Add Truck Routes

for i in range(0, truckRoute.shape[0] - 1):
    popup = "{i} --> {j}"
    
    coordinates = [[truckRoute["X"][i], truckRoute["Y"][i]], [truckRoute["X"][i+1], truckRoute["Y"][i+1]]]
    route = client.directions(
    coordinates=coordinates,
    profile='driving-car',
    format='geojson',
    validate=False,)
    plugins.AntPath(locations=[list(reversed(coord)) 
                           for coord in 
                           route['features'][0]['geometry']['coordinates']], 
                           tooltip = popup.format(i = truckRoute["Index"][i], j = truckRoute["Index"][i+1]), linecap ="round").add_to(m)
   
m

In [7]:
droneRoute

,DroneIndex,iIndex,jIndex,kIndex,iX,iY,jX,jY,kX,kY,Launch[i],Arrival[j],Launch[j],Retrieval[k]


In [8]:
# Add Drone Routes
colorsForDroneRoutes = ["red", "purple", "yellow", "green", "orange"]

for rowNumber in range(0, droneRoute.shape[0]):
    i = (droneRoute["iY"][rowNumber], droneRoute["iX"][rowNumber])
    j = (droneRoute["jY"][rowNumber], droneRoute["jX"][rowNumber])
    k = (droneRoute["kY"][rowNumber], droneRoute["kX"][rowNumber])
    
    color = colorsForDroneRoutes[droneRoute["DroneIndex"][rowNumber]]
    
    ijRouteToolTip = "{i} --> {j} | LaunchTime[{i}]: {launch} / ArrivalTime[{j}]: {arrival}"
    jkRouteToolTip = "{j} --> {k} | LaunchTime[{j}]: {launch} / RetrievalTime[{k}]: {retrieval}"
    plugins.AntPath(locations = [i,j], color=color, tooltip = ijRouteToolTip.format(i = droneRoute["iIndex"][rowNumber],
                                                                                   j = droneRoute["jIndex"][rowNumber],
                                                                                   launch = droneRoute["Launch[i]"][rowNumber],
                                                                                   arrival = droneRoute["Arrival[j]"][rowNumber])).add_to(m)
    plugins.AntPath(locations = [j,k], color=color, tooltip = jkRouteToolTip.format(j = droneRoute["jIndex"][rowNumber],
                                                                                   k = droneRoute["kIndex"][rowNumber],
                                                                                   launch = droneRoute["Launch[j]"][rowNumber],
                                                                                   retrieval = droneRoute["Retrieval[k]"][rowNumber])).add_to(m)
m    

In [9]:
m.save('map.html')

In [10]:
route["features"][0]["properties"]["summary"]

{'distance': 45455.3, 'duration': 2600.4}